In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(f'train set은 {train.shape[1]} 개의 feature를 가진 {train.shape[0]} 개의 데이터 샘플로 이루어져 있습니다.')
train.head()

train set은 15 개의 feature를 가진 1350 개의 데이터 샘플로 이루어져 있습니다.


,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


In [ ]:
train['target'].value_counts()

155000    15
140000    14
145000    13
170000    12
135000    12
          ..
126175     1
369900     1
374000     1
224500     1
256000     1
Name: target, Length: 619, dtype: int64

## 결측치 확인

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


# 데이터 설명 방정식

In [ ]:
train.columns

Index(['id', 'Overall Qual', 'Gr Liv Area', 'Exter Qual', 'Garage Cars',
       'Garage Area', 'Kitchen Qual', 'Total Bsmt SF', '1st Flr SF',
       'Bsmt Qual', 'Full Bath', 'Year Built', 'Year Remod/Add',
       'Garage Yr Blt', 'target'],
      dtype='object')

In [ ]:
Y = train[['target']]
X = train.drop(['target'],axis=1)

In [ ]:
X

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,1346,6,1756,Gd,2,422,TA,872,888,Ex,2,1996,1997,1996
1346,1347,9,2748,Gd,3,850,Ex,1850,1850,Ex,2,2006,2006,2006
1347,1348,5,1214,TA,1,318,TA,1214,1214,TA,2,1967,1967,1967
1348,1349,5,894,TA,2,440,TA,864,894,Gd,1,1974,1974,1974


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
X['Exter Qual']= encoder.fit_transform(X['Exter Qual'])
X['Kitchen Qual']= encoder.fit_transform(X['Kitchen Qual'])
X['Bsmt Qual']= encoder.fit_transform(X['Bsmt Qual'])

In [ ]:
test['Exter Qual']= encoder.fit_transform(test['Exter Qual'])
test['Kitchen Qual']= encoder.fit_transform(test['Kitchen Qual'])
test['Bsmt Qual']= encoder.fit_transform(test['Bsmt Qual'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

clf1 = LogisticRegression()
clf2 = RandomForestClassifier(n_estimators=100, random_state=1)
clf3 = DecisionTreeClassifier()

model = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='hard')

model.fit(X,Y)
pred = model.predict(test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['target']=pred
submission.to_csv('pred.csv',index=False)